In [1]:
import os, json, random
from datasets import load_dataset
from enum import Enum
from openai import OpenAI

In [2]:
class RoleType(Enum):
    USER = 'user'
    SYSTEM = 'system'
    ASSISTANT = 'assistant'

class Role(object):
    def __init__(self, role_type:RoleType, content):
        self.role = role_type.value
        self.content = content
        self.value = {'role': self.role, 'content':self.content }

class Message(object):
    def __init__(self, user_content, system_content, assistant_content):
        self.user_role = Role(role_type=RoleType.USER , content=user_content)
        self.system_role = Role(role_type=RoleType.SYSTEM , content=system_content)
        self.assistant_role = Role(role_type=RoleType.ASSISTANT, content=assistant_content)
        self.message = {'messages':[self.system_role.value, self.user_role.value, self.assistant_role.value, ]}

In [3]:
#Change Path
dataset = load_dataset("Amod/mental_health_counseling_conversations", split = 'train')

In [4]:
context = dataset[152]['Context']
response = dataset[152]['Response']
system_content = "You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user’s emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It’s important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibility is to create a secure space for users to express themselves and reflect. Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. Above all, prioritize their well-being and safety."
message_obj = Message(user_content=context, system_content=system_content, assistant_content=response)

print(message_obj.message)

{'messages': [{'role': 'system', 'content': 'You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user’s emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It’s important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsib

In [5]:
sampled_dataset = random.choices(dataset, k=100)
train_dataset = []

print(sampled_dataset[1])

for row in sampled_dataset:
    message_obj = Message(user_content=row['Context'], system_content=system_content, assistant_content=row['Response'])
    train_dataset.append(message_obj.message)

print(train_dataset[1])

{'Context': "I've worked with this guy for about three years now, and I knew the first time I saw him that I was attracted. Over time, he became a very dear friend, and we talked about our relationship problems, family, dreams, and so on. There was always flirtation with us, and one day pretty recently, it progressed to making out. We agreed to keep it quiet, and we did, but in the back of my mind, I hoped it would become something more. It didn't, and three months later, I'm still not over it.\r\n\r\nI'm trying so hard to maintain the friendship because that's what he wants. We're no longer in the same office, which I thought would help me get over these feelings, but we still talk about work, and he's constantly touching me, so I'm still very drawn to him. He's a hugger and such a sweet guy, so I find myself fantasizing all the time about what could have been and what could still be, but he's clearly not interested. He's younger than me and prefers model types, but I see him checking

In [6]:
def save_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for row in data:
            line = json.dumps(row)
            file.write(line + '\n')

#Change path
training_data_path = 'data/train.jsonl'
save_to_jsonl(train_dataset[:-5], training_data_path)

#Change path
validation_data_path = 'data/validation.jsonl'
save_to_jsonl(train_dataset[-5:], validation_data_path)

In [7]:
training_data = open(training_data_path, "rb")
validation_data = open(validation_data_path, "rb")

In [ ]:
api_key = '<HERE-LIES-KEY>'
client = OpenAI(api_key = api_key)

In [11]:
training_response = client.files.create(file=training_data, purpose="fine-tune")
training_file_id = training_response.index

validation_response = client.files.create(file=validation_data, purpose="fine-tune")
validation_file_id = validation_response.index

print("Training file id:", training_file_id)
print("Validation file id:", validation_file_id)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: <sk-proj**********************************************************************************************************************************************************YIA>. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
# create a fine-tuning job
response = client.fine_tuning.jobs.create(
    training_file = training_file_id
    model="gpt-3.5-turbo",
    suffix="my-test-model",
    validation_file=validation_file_id
)

job_id = response.id
print(response)

In [ ]:
# retrieve the job status
job_id = response.id

job_status = client.fine_tuning.jobs.retrieve(job_id)
print(job_status)

In [ ]:
#Test with samples
system_message = """You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user's emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It's important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibility is to create a secure space for users to express themselves and reflect. Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. Above all, prioritize their well-being and safety."""

messages = []
messages.append({"role": "system", "content": system_message})
user_message = "Every winter I find myself getting sad because of the weather. How can I fight this?"
messages.append({"role": "user", "content": user_message})


In [ ]:
completion = client.chat.completions.create(
    model= response.model,
    messages=messages
)
print(completion.choices[0].message)

In [ ]:
completion = client.chat.completions.create(
    model= "gpt-3.5-turbo",
    messages=messages
)
print(completion.choices[0].message)